# Given the CSV of labels information
## Restructure for consistency with column's in Galen's scripts for ease of cropping

```
gsv_panorama_id,sv_image_x,sv_image_y,label_type_id,photagrapher_heading,heading,is_researcher

stxXyCKAbd73DmkM2vsIHA,9676,-455,1,50.4137916565,253.375,t
```

## Comments
* Need to merge with metadata file to get the gsv_panorama_id
* Want to keep an image id as well
* sv_image_x and sv_image_y 
* 

In [1]:
import pandas as pd
import os
import s3fs # for reading from S3FileSystem
import json # for working with JSON files 

pd.set_option('max_colwidth', -1)

# Load current Labels CSV

In [2]:
SAGEMAKER_PATH = r'/home/ec2-user/SageMaker'
df_labels = pd.read_csv('all_labels_bool.csv')
print(df_labels.shape)
print(df_labels.dtypes)
df_labels.head()

(3440, 14)
filename                   object
file_size                  int64 
file_attributes            object
region_count               int64 
region_id                  int64 
region_shape_attributes    object
region_attributes          object
img_id                     int64 
present_ramp               bool  
missing_ramp               bool  
obstacle                   bool  
surface_prob               bool  
no_sidewalk                bool  
null                       bool  
dtype: object


,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes,img_id,present_ramp,missing_ramp,obstacle,surface_prob,no_sidewalk,null
0,680_45.jpg,49558,{},3,0,"{""name"":""point"",""cx"":108,""cy"":389}","{""Present Curb Ramp"":""1\n""}",680,True,False,False,False,False,False
1,680_45.jpg,49558,{},3,1,"{""name"":""point"",""cx"":160,""cy"":389}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False
2,680_45.jpg,49558,{},3,2,"{""name"":""point"",""cx"":607,""cy"":453}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False
3,680_135.jpg,51194,{},6,0,"{""name"":""point"",""cx"":18,""cy"":475}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False
4,680_135.jpg,51194,{},6,1,"{""name"":""point"",""cx"":245,""cy"":413}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False


# Limit to where there's exactly 1 region_attribute on a point

In [8]:
# Find the roughly 26 rows where there are multiple attributes in a given tag
df_limit = df_labels.loc[~df_labels['region_attributes'].str.contains(',')]
# Find where no tags were given to an image
df_limit = df_labels.loc[df_labels['region_attributes'] != '{}']
df_limit.shape

(2877, 14)

## Parse out x and y coordinates
## Name as sv_image_x and sv_image_y

In [9]:
df_coor = df_limit['region_shape_attributes'].str.extract('{"name":"point","cx":(.*),"cy":(.*)}')
df_coor.columns = ['sv_image_x', 'sv_image_y']
df_coor['sv_image_x'] = pd.to_numeric(df_coor['sv_image_x'])
df_coor['sv_image_y'] = pd.to_numeric(df_coor['sv_image_y'])
df_coor.head()

,sv_image_x,sv_image_y
0,108,389
1,160,389
2,607,453
3,18,475
4,245,413


In [10]:
df_coor.describe()

,sv_image_x,sv_image_y
count,2877.000000,2877.000000
mean,315.975669,410.168231
std,152.431051,47.565286
min,2.000000,325.000000
25%,209.000000,377.000000
50%,317.000000,404.000000
75%,421.000000,430.000000
max,634.000000,634.000000


In [ ]:
coor_cols = ['sv_image_x', 'sv_image_y']
df_limit[coor_cols] = 